In [1]:

import time
import math
import json
import pickle
import warnings
import statistics

import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from os import listdir
from os.path import join
from os.path import isfile

from itertools import combinations

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.exceptions import ConvergenceWarning


from libsvm.svmutil import svm_problem
from libsvm.svmutil import svm_parameter
from libsvm.svmutil import svm_train
from libsvm.svmutil import svm_predict
from libsvm.svmutil import evaluations

from Function import svm_function

# Filter out ConvergenceWarning, RuntimeWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [30]:
dir_path = "data/RBP_eclip/output/"
data_path = "data/RBP_eclip/All/merge/"

perf_list = []

perf_file_list = ["All___noneclip_merge__RBPpred__RBPPWM_10-3_or___esvm_f10_e512_s10_p8_t1000.json",
                  "All___eclip_merge__RBPpred__RBPPWM_10-3_or___esvm_f10_e512_s10_p8_t1000.json"]
x_file_list = ["noneclip_merge__RBPpred__RBPPWM_10-3_or.npy",
               "eclip_merge__RBPpred__RBPPWM_10-3_or.npy"]
y_file_list = ["noneclip__y.npy",
               "eclip__y.npy"]

# perf_file_list = ["All___ensembl_non-RBPeclip__model_predict__trian_cutoff___esvm_f10_e512_s10_p8_t1000.json",
#                   "All___ensembl_non-RBPeclip__model_predict_proba___esvm_f10_e512_s10_p8_t1000.json",
#                   "All___ensembl_RBPeclip_model__has_loc_predict___esvm_f10_e512_s10_p8_t1000.json",
#                   "All___ensembl_RBPeclip_model__has_loc_predict_proba___esvm_f10_e512_s10_p8_t1000.json",
#                   "All___RBP_eclip_has_non__predict___esvm_f10_e512_s10_p8_t1000.json",
#                   "All___RBP_eclip_has_non__predict_proba___esvm_f10_e512_s10_p8_t1000.json"]
# x_file_list = ["ensembl_non-RBPeclip__model_predict__trian_cutoff.npy",
#                "ensembl_non-RBPeclip__model_predict_proba.npy",
#                "ensembl_RBPeclip_model__has_loc_predict_proba.npy",
#                "ensembl_RBPeclip_model__has_loc_predict.npy",
#                "RBP_eclip_has_non__predict.npy",
#                "RBP_eclip_has_non__predict_proba.npy"]
# y_file_list = ["ensembl_non-RBPeclip__loc.npy",
#                "ensembl_non-RBPeclip__loc.npy",
#                "ensembl_RBPeclip_model__has_loc__y.npy",
#                "ensembl_RBPeclip_model__has_loc__y.npy",
#                "RBP_eclip_has_non__y.npy",
#                "RBP_eclip_has_non__y.npy"]

for i in range(len(y_file_list)):

    file_list = perf_file_list[i].split("___")
    print(file_list)
    f = open(dir_path + perf_file_list[i])
    perf = json.load(f)
    f.close()
    x = np.load(data_path + x_file_list[i])
    u, c = np.unique(np.load(data_path + y_file_list[i]), return_counts=True)
    perf_list.append([file_list[1], x.shape[1], x.shape[0], c[np.where(u == 0)[0][0]], c[np.where(u == 1)[0][0]],
                      file_list[2],
                        perf['avg Accy'], perf['std Accy'],
                        perf['avg Recall'], perf['std Recall'],
                        perf['avg Prec'], perf['std Prec'],
                        perf['avg Spec'], perf['std Spec'],
                        perf['avg Npv'], perf['std Npv'],
                        perf['avg F1sc'], perf['std F1sc'],
                        perf['avg AUROC'], perf['std AUROC']
                        ])

['All', 'noneclip_merge__RBPpred__RBPPWM_10-3_or', 'esvm_f10_e512_s10_p8_t1000.json']
['All', 'eclip_merge__RBPpred__RBPPWM_10-3_or', 'esvm_f10_e512_s10_p8_t1000.json']


In [31]:
perf_file_path = "data/RBP_eclip/output/feature_selection__noneclip_merge__e512u25f10s10t1000.json"
perf_file = "feature_selection__noneclip_merge__e512u25f10s10t1000.json"
x_file = "data/RBP_eclip/All/merge/noneclip_merge__RBPpred__RBPPWM_10-3_or.npy"
y_file = "data/RBP_eclip/All/merge/noneclip__y.npy"

file_list = perf_file.split("__")
print(file_list)
f = open(perf_file_path)
perf = json.load(f)
f.close()
x = np.load(x_file)
u, c = np.unique(np.load(y_file), return_counts=True)

selected_features_num = sum(np.array(perf['selected_features']) == 'True')
perf = perf['subset_perf']
perf_list.append(['feature_selection ' + file_list[1], selected_features_num, x.shape[0], c[np.where(u == 0)[0][0]], c[np.where(u == 1)[0][0]],
                    file_list[2],
                    perf['avg Accy'], perf['std Accy'],
                    perf['avg Recall'], perf['std Recall'],
                    perf['avg Prec'], perf['std Prec'],
                    perf['avg Spec'], perf['std Spec'],
                    perf['avg Npv'], perf['std Npv'],
                    perf['avg F1sc'], perf['std F1sc'],
                    perf['avg AUROC'], perf['std AUROC']
                    ])

['feature_selection', 'noneclip_merge', 'e512u25f10s10t1000.json']


In [32]:
df = pd.DataFrame(perf_list, columns=["name", "Feature num", "nums", "count_0", "count_1", "hp",
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ])

In [35]:
f = open(perf_file_path)
perf = json.load(f)
f.close()
x = np.load(x_file)

# selected_features_num = sum(np.array(perf['selected_features']) == 'True')

In [39]:
sum(np.array(perf['selected_features'])[:154] == 'True'), sum(np.array(perf['selected_features'])[154:] == 'True') 

(76, 22)

In [33]:
df

,name,Feature num,nums,count_0,count_1,hp,avg Accy,std Accy,avg Recall,std Recall,avg Prec,std Prec,avg Spec,std Spec,avg Npv,std Npv,avg F1sc,std F1sc,avg AUROC,std AUROC
0,noneclip_merge__RBPpred__RBPPWM_10-3_or,205,594,95,499,esvm_f10_e512_s10_p8_t1000.json,0.464166,0.113508,0.426327,0.147754,0.864958,0.061209,0.656667,0.174374,0.182693,0.047926,0.556386,0.146780,0.558009,0.100398
1,eclip_merge__RBPpred__RBPPWM_10-3_or,205,2237,269,1968,esvm_f10_e512_s10_p8_t1000.json,0.758183,0.023253,0.755099,0.023834,0.962120,0.017002,0.780627,0.101426,0.303581,0.031641,0.845853,0.016396,0.851919,0.048062
2,feature_selection noneclip_merge,98,594,95,499,e512u25f10s10t1000.json,0.626361,0.037818,0.629102,0.048918,0.895799,0.028659,0.613333,0.113942,0.238711,0.028384,0.737755,0.034676,0.669131,0.076546


In [34]:
df.to_csv("data/RBP_eclip/output/All_perf_0704.csv", index=False)

In [ ]:
dir_path = "data/Pse_in_One2/my_DB/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

perf_list = []
for file_name in onlyfiles:
    if ("json" in file_name) and not ".dvc" in file_name and "All" in file_name:
        file_list = file_name.split("__")
        print(file_list)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        
        
        u, c = np.unique(np.load("data/Pse_in_One2/my_DB/train/%s__%s__train_y.npy" % (file_list[0], file_list[1])), return_counts=True)
        perf_list.append([file_list[1], c[np.where(u == 0)[0][0]], c[np.where(u == 1)[0][0]], file_list[3][:-5],
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC']
                          ])
        
df = pd.DataFrame(perf_list, columns=["name", "count_0", "count_1", "hp", 
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ])#.sort_values("name").to_csv("data/Pse_in_One2/my_DB/output/v230509__pref.csv", index=False)

# rbp_df = pd.DataFrame([])
# rbp_df["name"] = rbp_list


In [24]:
dir_path = "data/Pse_in_One2/my_DB/train/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

rbp_list = []
for file_name in onlyfiles:
    if ("__train_y.npy" in file_name) and not ".dvc" in file_name:
        file_list = file_name.split("__")
        rbp_list.append(file_list[1])

In [25]:
dir_path = "data/Pse_in_One2/my_DB/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

perf_list = []
for file_name in onlyfiles:
    if ("json" in file_name) and not ".dvc" in file_name and "All" in file_name:
        file_list = file_name.split("__")
        print(file_list)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        
        
        u, c = np.unique(np.load("data/Pse_in_One2/my_DB/train/%s__%s__train_y.npy" % (file_list[0], file_list[1])), return_counts=True)
        perf_list.append([file_list[1], c[np.where(u == 0)[0][0]], c[np.where(u == 1)[0][0]], file_list[3][:-5],
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC']
                          ])
        
df = pd.DataFrame(perf_list, columns=["name", "count_0", "count_1", "hp", 
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ])#.sort_values("name").to_csv("data/Pse_in_One2/my_DB/output/v230509__pref.csv", index=False)

# rbp_df = pd.DataFrame([])
# rbp_df["name"] = rbp_list


['ensembl_has_rbpelic', 'NSUN2', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'HLTF', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'ZC3H11A', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'SF3B4', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'NIPBL', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'AKAP8L', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'SAFB2', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'PABPC4', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'DROSHA', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'SERBP1', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'PPIG', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'ABCF1', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'MATR3', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'FXR1', 'train_y', 'svm_p4_e512_f10.json']
['ensembl_has_rbpelic', 'GEMIN5', 'train_y', 

In [26]:
pd.merge(rbp_df, df, how="left", on="name").sort_values("name").to_csv("data/Pse_in_One2/my_DB/output/v230613__cutoff_pref.csv", index=False)

## optuna perf

In [11]:
storage = optuna.storages.JournalStorage(
        optuna.storages.JournalFileStorage("data/RBP_eclip/output/ensembl__optuna.log"),
    )

/tmp/ipykernel_8311/943740313.py:1: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage = optuna.storages.JournalStorage(


In [12]:
f_col = []
for i in range(205):
    f_col.append("feature_%s" % i)

In [34]:
study_list = [
    "ensembl__merge_RBP_PWM__fs",
    "ensembl__merge_RBP_PWM__random_sampler__fs",
    "ensembl__merge_RBP_PWM10-4__random_sampler__fs"
]
# x_list = [
#     "data/RBP_eclip/All/merge/ensembl__merge__RBPpred__RBPPWM_10-3_or.npy",
#     "data/RBP_eclip/All/merge/ensembl__merge__RBPpred__RBPPWM_10-4_or.npy"
# ]
y_list = [
    "data/RBP_eclip/All/merge/ensembl__y.npy",
    "data/RBP_eclip/All/merge/ensembl__y.npy",
    "data/RBP_eclip/All/merge/ensembl__y.npy"
]
output_list = [
    "data/RBP_eclip/output/ensembl__merge_RBP_PWM__fs.json",
    "data/RBP_eclip/output/ensembl__merge_RBP_PWM__random_sampler__fs.json",
    "data/RBP_eclip/output/ensembl__merge_RBP_PWM10-4__random_sampler__fs.json"
]

perf_list = []
for i in range(len(study_list)):
    study = optuna.load_study(study_name=study_list[i], storage=storage)
    best_params = study.best_params
    
    select_feature = [best_params[f] for f in f_col]
    
    f = open(output_list[i])
    perf = json.load(f)
    f.close()
    
    u, c = np.unique(np.load(y_list[i]), return_counts=True)
    perf_list.append([study_list[i], sum(select_feature), c[np.where(u == 0)[0][0]], c[np.where(u == 1)[0][0]],
                        perf['avg Accy'], perf['std Accy'],
                        perf['avg Recall'], perf['std Recall'],
                        perf['avg Prec'], perf['std Prec'],
                        perf['avg Spec'], perf['std Spec'],
                        perf['avg Npv'], perf['std Npv'],
                        perf['avg F1sc'], perf['std F1sc'],
                        perf['avg AUROC'], perf['std AUROC']
                        ])
    
df = pd.DataFrame(perf_list, columns=["name", "feature", "count_0", "count_1", 
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ])

In [37]:
df.to_csv("data/RBP_eclip/output/feature_selection___ensembl__merge.csv", index=False)

## optuna study

In [2]:
storage = optuna.storages.JournalStorage(
        optuna.storages.JournalFileStorage("data/RBP_eclip/output/ensembl__optuna.log"),
    )
# [(storage.get_study_name_from_id(temp_study._study_id), temp_study._study_id) for temp_study in storage.get_all_studies()]

/tmp/ipykernel_8311/2765332447.py:1: ExperimentalWarning: JournalStorage is experimental (supported from v3.1.0). The interface can change in the future.
  storage = optuna.storages.JournalStorage(


In [22]:
[(storage.get_study_name_from_id(temp_study._study_id), temp_study._study_id) for temp_study in storage.get_all_studies()]

[('ensembl__merge_RBP_PWM__fs', 0),
 ('ensembl__merge_RBP_PWM', 1),
 ('ensembl__merge_RBP_PWM__t100000', 2),
 ('ensembl__merge_RBP_PWM__size', 3),
 ('ensembl__merge_RBP_PWM__random_sampler__fs', 5),
 ('ensembl__merge_RBP_PWM10-4__random_sampler__fs', 9),
 ('ensembl__merge_RBP_PWM10-3__random_sampler__fs', 10)]

In [23]:
study = optuna.load_study(study_name="ensembl__merge_RBP_PWM__fs", storage=storage)
best_params = study.best_params

In [28]:
study.best_trial

FrozenTrial(number=532, state=TrialState.COMPLETE, values=[0.8040063497139878], datetime_start=datetime.datetime(2023, 7, 5, 16, 18, 39, 661942), datetime_complete=datetime.datetime(2023, 7, 5, 16, 34, 3, 959489), params={'feature_0': False, 'feature_1': True, 'feature_2': True, 'feature_3': False, 'feature_4': False, 'feature_5': True, 'feature_6': True, 'feature_7': False, 'feature_8': True, 'feature_9': False, 'feature_10': False, 'feature_11': False, 'feature_12': False, 'feature_13': False, 'feature_14': False, 'feature_15': False, 'feature_16': True, 'feature_17': True, 'feature_18': True, 'feature_19': False, 'feature_20': True, 'feature_21': True, 'feature_22': True, 'feature_23': False, 'feature_24': False, 'feature_25': False, 'feature_26': False, 'feature_27': True, 'feature_28': True, 'feature_29': True, 'feature_30': True, 'feature_31': True, 'feature_32': False, 'feature_33': False, 'feature_34': False, 'feature_35': False, 'feature_36': False, 'feature_37': True, 'featur

In [24]:
f_col = []
for i in range(205):
    f_col.append("feature_%s" % i)

In [25]:
select_feature = [best_params[f] for f in f_col]

In [26]:
temp = {}
for k in best_params.keys():
    if not k in f_col:
        temp[k] = best_params[k]

In [30]:
# data/RBP_eclip/All/merge/ensembl__merge__RBPpred__RBPPWM_10-3_or.npy
# data/RBP_eclip/All/merge/ensembl__y.npy
x = np.load("data/RBP_eclip/All/merge/ensembl__merge__RBPpred__RBPPWM_10-3_or.npy")
y = np.load("data/RBP_eclip/All/merge/ensembl__y.npy")
perf_json = svm_function.cv_esvm_perf(data_x=x[:, select_feature], 
                                      data_y=y,
                                      size=10,
                                      max_iter=1000,
                                      log=False,
                                      fold=10,
                                      **temp
                                      )

In [31]:
with open('%s.json' % ("data/RBP_eclip/output/ensembl__merge_RBP_PWM__fs"), 'w') as fp:
    json.dump(perf_json, fp)

## predict_proba predict 

In [21]:
def pred_proba_perf(train_proba, train_y, test_proba, test_y, auto_threshold=False):
    if auto_threshold:
        print("Auto_threshold")
    
    acc_array = []
    recall_array = []
    prec_array = []
    spec_array = []
    npv_array = []
    f1sc_array = []
    auroc_array = []
    for i in range(len(test_y)):
        y_train = np.array(train_y[i])
        y_train_proba = np.array(train_proba[i])[:, 1]
        
        y_test = np.array(test_y[i])
        y_test_proba = np.array(test_proba[i])[:, 1]
        
        threshold = 0.5
        if auto_threshold:
            fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_proba)
            diff = []
            for i in range(len(thresholds)):
                pred = np.where(y_train_proba > thresholds[i], 1, 0)
                tn, fp, fn, tp = confusion_matrix(y_train, pred).ravel()
                
                # recall = tp / (fn + tp)
                # prec = tp / (fp + tp)
                # spec = tn / (tn + fp)
                # npv = tn / (tn + fn)
                # diff.append(recall * prec)
                
                # recall = (tp / (fn + tp)) ** 2
                # prec = (tp / (fp + tp)) ** 2
                # spec = (tn / (tn + fp)) ** 2
                # npv =  (tn / (tn + fn)) ** 2
                # diff.append(recall + prec + spec + npv)
                
                diff.append(metrics.roc_auc_score(y_train, pred))

            diff = np.array(diff)

            diff[np.isnan(diff)] = 0
            threshold = thresholds[np.argmax(diff)]
            
        y_test_pred = np.where(y_test_proba > threshold, 1, 0)
        try:
            roc_score = metrics.roc_auc_score(y_test, y_test_pred)
            auroc_array.append(roc_score)
        except:
            auroc_array.append(0.5)
        
        
        tn, fp, fn, tp = confusion_matrix(y_test, y_test_pred).ravel()
        
        acc_array.append((tn + tp) / (tn + fp + fn + tp))
        recall_array.append(tp / (fn + tp))
        prec_array.append(tp / (fp + tp))
        spec_array.append(tn / (tn + fp))
        npv_array.append(tn / (tn + fn))
        f1sc_array.append(2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
    
    json_dict = {
        "fold Accy": acc_array,
        "avg Accy": sum(acc_array) / len(acc_array),
        "std Accy": np.std(acc_array),
        "fold Recall": recall_array,
        "avg Recall": sum(recall_array) / len(recall_array),
        "std Recall": np.std(recall_array),
        "fold Prec": prec_array,
        "avg Prec": sum(prec_array) / len(prec_array),
        "std Prec": np.std(prec_array),
        "fold Spec": spec_array,
        "avg Spec": sum(spec_array) / len(spec_array),
        "std Spec": np.std(spec_array),
        "fold Npv": npv_array,
        "avg Npv": sum(npv_array) / len(npv_array),
        "std Npv": np.std(npv_array),
        "fold F1sc": f1sc_array,
        "avg F1sc": sum(f1sc_array) / len(f1sc_array),
        "std F1sc": np.std(f1sc_array),
        "fold AUROC": auroc_array,
        "avg AUROC": sum(auroc_array) / len(auroc_array),
        "std AUROC": np.std(auroc_array),
    }
    
    return json_dict
 

In [4]:
dir_path = "data/Pse_in_One2/my_DB/output/predict_output/"
output_file = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [23]:
dir_path = "data/Pse_in_One2/my_DB/output/predict/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

rbp_list = []
for file_name in onlyfiles:
    if ("__train_y__svm_p4_e512_f10.json" in file_name) and not ".dvc" in file_name and not file_name in output_file:
        print(file_name)
        f = open(dir_path + file_name)
        pred_proba = json.load(f)
        f.close()
        
        f = open(pred_proba["hp"])
        hp = json.load(f)
        f.close()
        # if hp["avg Recall"] == 1:
        ppp = pred_proba_perf(pred_proba["train_proba"], 
                            pred_proba["train_y"], 
                            pred_proba["test_proba"], 
                            pred_proba["test_y"], 
                            auto_threshold=True)
        
        ppp["input"] = pred_proba["input"]
        ppp["label"] = pred_proba["label"]
        ppp["hp"] = pred_proba["hp"]
        
        with open('data/Pse_in_One2/my_DB/output/predict_output/%s' % (file_name), 'w') as fp:
            json.dump(ppp, fp)
        # for key, val in ppp.items():
        #     if "avg" in key:
        #         print(key, val, hp[key])


ensembl_has_rbpelic__EIF3D__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__XRCC6__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__DDX3X__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__SRSF9__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__TAF15__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__HNRNPC__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__AQR__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__ZNF622__train_y__svm_p4_e512_f10.json
Auto_threshold
ensembl_has_rbpelic__FUS__train_y__svm_p4_e512_f10.json
Auto_threshold


`for x in *.json.json; do  mv $x ${x%.*.*}.json; done;`

### recall, prec, spec, npv ** 2
NSUN2
avg Accy 0.9690926777019493 0.9689496685656438
avg Recall 0.39391304347826084 0.5556107660455487
avg Prec 0.9611648941922775 0.7568310802548311
avg Spec 0.9991699251543869 0.9905675744898831
avg Npv 0.9692587113568318 0.9770818341530194
avg F1sc 0.5578832101838608 0.6393702828424033
avg AUROC 0.696541484316324 0.8810535817970357

NIPBL
avg Accy 0.8211532270920431 0.2179993109229005
avg Recall 0.599671052631579 1.0
avg Prec 0.5893865216348267 0.2179993109229005
avg Spec 0.882893650299784 0.0
avg Npv 0.8879207871503813 nan
avg F1sc 0.5935713000824612 0.357962939469001
avg AUROC 0.7412823514656814 0.792660783091631
### auroc
NSUN2
avg Accy 0.9694512549120098 0.9689496685656438
avg Recall 0.4127536231884058 0.5556107660455487
avg Prec 0.944960647656037 0.7568310802548311
avg Spec 0.9985663223199295 0.9905675744898831
avg Npv 0.9701775512149207 0.9770818341530194
avg F1sc 0.5696783483784433 0.6393702828424033
avg AUROC 0.7056599727541677 0.8810535817970357

NIPBL
avg Accy 0.8423090253672936 0.2179993109229005
avg Recall 0.525 1.0
avg Prec 0.6790399997573106 0.2179993109229005
avg Spec 0.9307659835686476 0.0
avg Npv 0.8755203267619791 nan
avg F1sc 0.5916631645749215 0.357962939469001
avg AUROC 0.7278829917843238 0.792660783091631

---

In [6]:
y = np.load("data/merge_data/seq_rbp/pse_in_one2__rbp_10-3_-log10__train__y.npy")

In [7]:
perf_list = []

In [20]:
dir_path = "data/RBP_eclip/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

for file_name in onlyfiles:
    if ("hpo_svm_train__" in file_name) and ("_hasNu.json" in file_name) and not ".dvc" in file_name:
        file_name_list = file_name.split("__")
        print(file_name_list)
        file_str = file_name_list[1]
        file_hp = file_name_list[-1][:-len(".json")]
        
        print(file_str)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        num = np.load("data/RBP_eclip/train/" + file_str + ".npy").shape[1]
        perf_list.append([file_str, num, file_hp,
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC']
                          ])

['hpo_svm_train', 'x', 'esvm_e512_p10_f10_s10_t1000_hasNu.json']
x
['hpo_svm_train', 'x', 'esvm_e512_p10_f10_s10_t10000_hasNu.json']
x
['hpo_svm_train', 'x', 'esvm_e1024_p10_f10_s10_t1000_hasNu.json']
x
['hpo_svm_train', 'x', 'esvm_e1024_p10_f10_s10_t10000_hasNu.json']
x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e512_p10_f10_s10_t1000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e512_p10_f10_s10_t10000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e1024_p10_f10_s10_t1000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e1024_p10_f10_s10_t10000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e512_p10_f10_s20_t10000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e512_p10_f10_s20_t1000_hasNu.json']
pseinone_rbpeclip_x
['hpo_svm_train', 'pseinone_rbpeclip_x', 'esvm_e1024_p10_f10_s20_t10000_hasNu.json']
pseinone_rbpeclip_x


In [15]:
dir_path = "data/merge_data/seq_rbp_output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

for file_name in onlyfiles:
    if ("hpo_svm_train__" in file_name) and ("_hasNu.json" in file_name) and not "pse" in file_name and not ".dvc" in file_name:
        file_name_list = file_name.split("__")
        print(file_name_list)
        # file_str = file_name_list[1] + "__" + file_name_list[2]
        file_str = file_name_list[1]
        file_hp = file_name_list[-1][:-len(".json")]
        
        print(file_str)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        # num = np.load("data/merge_data/seq_rbp/transcript2pse_in_one__train__x.npy").shape[1]
        num = np.load("data/merge_data/seq_rbp/" + file_str + "__train__x.npy").shape[1]
        perf_list.append([file_str, num, file_hp,
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC']
                          ])

['hpo_svm_train', 'rbp_10-3_-log10_utest1e-4', 'esvm_e512_p10_f10_s10_t1000_hasNu.json']
rbp_10-3_-log10_utest1e-4
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-5', 'esvm_e512_p10_f10_s10_t1000_hasNu.json']
rbp_10-3_-log10_utest1e-5
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-5', 'esvm_e512_p10_f10_s10_t10000_hasNu.json']
rbp_10-3_-log10_utest1e-5
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-5', 'esvm_e1024_p10_f10_s10_t1000_hasNu.json']
rbp_10-3_-log10_utest1e-5
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-4', 'esvm_e512_p10_f10_s10_t10000_hasNu.json']
rbp_10-3_-log10_utest1e-4
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-4', 'esvm_e1024_p10_f10_s10_t1000_hasNu.json']
rbp_10-3_-log10_utest1e-4
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-5', 'esvm_e1024_p10_f10_s10_t10000_hasNu.json']
rbp_10-3_-log10_utest1e-5
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-4', 'esvm_e1024_p10_f10_s10_t10000_hasNu.json']
rbp_10-3_-log10_utest1e-4
['hpo_svm_train', 'rbp_10-3_-log10_utest1e-5', 'esvm_e512_p10_f10_s10_t1

In [18]:
# dir_path = "data/merge_data/output/"
dir_path = "data/RBP-PWM/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]


for file_name in onlyfiles:
    if ("__esvm_p10_f10_s10__v230221.json" in file_name) and not ".dvc" in file_name:
        file_str = file_name[:-len("__esvm_p10_f10_s10__v230221.json")]
    # if ("__esvm_p10_f10_s10__v230118_v2.json" in file_name) and not ".dvc" in file_name:
    #     file_str = file_name[:-len("__esvm_p10_f10_s10__v230118_v2.json")]
        
        if "train__" in file_str:
            file_str = file_str.replace("train", "train_binary")
        print(file_str)
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        num = np.load("data/RBP-PWM/train/" + file_str + "__x.npy").shape[1]
        perf_list.append([file_str, num,
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg Npv'], perf['std Npv'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC'],
                          ])

train_binary__10-4__filter
train_binary__10-3__filter
train_-log10__10-3__filter
train_-log10__10-4__filter


In [21]:
pd.DataFrame(perf_list, columns=["esvm_s10", "nums", "hp", 
                                 "avg Accy", "std Accy",
                                 "avg Recall", "std Recall",
                                 "avg Prec", "std Prec",
                                 "avg Spec", "std Spec",
                                 "avg Npv", "std Npv",
                                 "avg F1sc", "std F1sc",
                                 "avg AUROC", "std AUROC"
                                 ]).sort_values("esvm_s10").to_csv("data/RBP_eclip/output/v230426__pref.csv", index=False)

In [5]:
np.load("data/linear_features/point/benchmark/train/k234p10nor2n3_train.npy").shape

(2264, 1008)

In [3]:
f = open("data/merge_data/output/Kmer_k3__DACC_lag2__SC-PseDNC-General_all_index_svmf10p4e512.json")
perf = json.load(f)
f.close()
["file_str", 
        perf['avg Accy'], perf['std Accy'],
        perf['avg Recall'], perf['std Recall'],
        perf['avg Prec'], perf['std Prec'],
        perf['avg Spec'], perf['std Spec'],
        perf['avg F1sc'], perf['std F1sc'],
        perf['avg AUROC'], perf['std AUROC'],
        ]

['file_str',
 0.7062645551932929,
 0.02728313642700328,
 0.724779861183052,
 0.027373539164752204,
 0.9191811524505985,
 0.013461697940586836,
 0.585483870967742,
 0.0729476979210487,
 0.8102467384324458,
 0.019711847903634495,
 0.6936555352450462,
 0.04558923440965074]

In [16]:
from numpy import average


dir_path = "data/linear_features/point/benchmark/output/"
onlyfiles = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

perf_list = []

for file_name in onlyfiles:
    if "_svmf10p4e512.json" in file_name and not ".dvc" in file_name and not "train" in file_name:
        
        file_str = file_name[:-5]
        f = open(dir_path + file_name)
        perf = json.load(f)
        f.close()
        perf_list.append([file_str, np.load("data/linear_features/point/benchmark/train/" + file_name[:-18] + "_train.npy").shape[1],
                          perf['avg Accy'], perf['std Accy'],
                          perf['avg Recall'], perf['std Recall'],
                          perf['avg Prec'], perf['std Prec'],
                          perf['avg Spec'], perf['std Spec'],
                          perf['avg F1sc'], perf['std F1sc'],
                          perf['avg AUROC'], perf['std AUROC'],
                          ])
        # npv_array = []
        # for i in range(len(perf['confusion matrix'])):
        #     tn, fp, fn, tp = np.array(perf['confusion matrix'][i][1]).flatten()
        #     acc = (tn + tp) / (tn + fp + fn + tp)
        #     recall = tp / (fn + tp)
        #     prec = (tp / (fp + tp))
        #     spec = (tn / (tn + fp))
        #     npv = (tn / (tn + fn))
        #     f1sc = (2 * (tp / (fn + tp)) * (tp / (fp + tp)) / ((tp / (fn + tp)) + (tp / (fp + tp))))
        #     npv_array.append(npv)
        # print(npv_array)
        # print(sum(npv_array)/ len(npv_array), np.std(npv_array))

In [18]:
pd.DataFrame(perf_list, columns=['file name', "num", 'avg Accy', 'std Accy', 'avg Recall', 'std Recall', 'avg Prec', 'std Prec', 'avg Spec', 'std Spec', 'avg F1sc', 'std F1sc', 'avg AUROC', 'std AUROC']).sort_values('file name').to_csv(dir_path + "svmf10p4e512_pref_v2.csv", index=False)